# Section 3

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocode
import folium # map rendering library
import requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


Create a link to download the data set

In [2]:
url = 'https://raw.githubusercontent.com/mirelmora/Coursera_Capstone/main/Clustering_Toronto_Week_3/Toronto.csv'
print('Data downloaded!')

Data downloaded!


Dataframe consist in 3 columns 

In [3]:
df = pd.read_csv(url)

Convert 'Not assigned' value to NaN and eliminated to dataframe

In [4]:
df['Borough'] = df['Borough'].replace('Not assigned', np.nan)

In [5]:
toronto_df = df.dropna()

In [6]:
url2 = 'https://raw.githubusercontent.com/mirelmora/Coursera_Capstone/main/Clustering_Toronto_Week_3/Geospatial_Coordinates.csv'

In [7]:
df_geo = pd.read_csv(url2)

In [8]:
df_geo.shape

(103, 3)

In [9]:
toronto_df = toronto_df.join(df_geo.set_index('Postal Code'), on="Postal Code")

In [10]:
toronto_df = toronto_df.reset_index()
toronto_df

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.753259,-79.329656
1,3,M4A,North York,Victoria Village,43.725882,-79.315572
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [12]:
toronto_df.drop(['index'], axis = 'columns', inplace = True)

In [13]:
toronto_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [14]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [15]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [16]:
CLIENT_ID = 'C1BCWFRQVGGYE43UIPCLSAQBAXHXBY2FQQDB1FW2R5S23Y4H' # your Foursquare ID https://www.google.com/?code=B3I3ZUDMT43EMRXO3NVS0FA4FMCLARYDK4REKPFRTOXETTG3#_=_
CLIENT_SECRET = '5N0TQE5DPN2AB5FOYD0VRLBG4WUMJCBWEPD3Q4LE2MTFORZL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 200

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C1BCWFRQVGGYE43UIPCLSAQBAXHXBY2FQQDB1FW2R5S23Y4H
CLIENT_SECRET:5N0TQE5DPN2AB5FOYD0VRLBG4WUMJCBWEPD3Q4LE2MTFORZL


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
# type your answer here
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(4891, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store


In [20]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,48,48,48,48,48,48
"Alderwood, Long Branch",25,25,25,25,25,25
"Bathurst Manor, Wilson Heights, Downsview North",34,34,34,34,34,34
Bayview Village,15,15,15,15,15,15
"Bedford Park, Lawrence Manor East",41,41,41,41,41,41
...,...,...,...,...,...,...
"Willowdale, Willowdale West",12,12,12,12,12,12
Woburn,9,9,9,9,9,9
Woodbine Heights,24,24,24,24,24,24


In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 334 uniques categories.


## 3. Analyze Each Neighborhood

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
manhattan_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods


In [23]:
toronto_onehot.shape

(4891, 335)

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.024390,0.0,0.0,...,0.02439,0.0,0.0,0.0,0.0,0.0,0.02439,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
94,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
95,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
96,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0


In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0  Chinese Restaurant  0.12
1       Shopping Mall  0.08
2          Restaurant  0.04
3              Bakery  0.04
4         Pizza Place  0.04


----Alderwood, Long Branch----
            venue  freq
0  Discount Store  0.12
1            Park  0.08
2     Pizza Place  0.08
3        Pharmacy  0.08
4      Print Shop  0.04


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0          Coffee Shop  0.06
1          Pizza Place  0.06
2                 Park  0.06
3                 Bank  0.06
4  Fried Chicken Joint  0.03


----Bayview Village----
                 venue  freq
0        Grocery Store  0.13
1                 Bank  0.13
2  Japanese Restaurant  0.13
3          Gas Station  0.13
4   Chinese Restaurant  0.07


----Bedford Park, Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.07
1           Coffee Shop  0.07
2                  Bank  0.05
3  Fast Food Restaurant  0.05
4        Sa

                  venue  freq
0     Indian Restaurant  0.08
1           Coffee Shop  0.05
2            Restaurant  0.04
3  Fast Food Restaurant  0.04
4                  Park  0.04


----Islington Avenue, Humber Valley Village----
           venue  freq
0       Pharmacy  0.17
1           Café  0.08
2           Bank  0.08
3  Shopping Mall  0.08
4    Golf Course  0.08


----Kennedy Park, Ionview, East Birchmount Park----
                  venue  freq
0    Chinese Restaurant  0.11
1        Discount Store  0.11
2           Coffee Shop  0.11
3  Fast Food Restaurant  0.07
4         Grocery Store  0.07


----Kensington Market, Chinatown, Grange Park----
                           venue  freq
0                           Café  0.10
1  Vegetarian / Vegan Restaurant  0.07
2                    Coffee Shop  0.05
3                            Bar  0.04
4                    Yoga Studio  0.03


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
               venue  freq
0  

4  Portuguese Restaurant  0.09


----West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale----
           venue  freq
0           Park  0.15
1    Pizza Place  0.15
2  Grocery Store  0.08
3           Bank  0.08
4          Hotel  0.08


----Westmount----
                       venue  freq
0                Gas Station  0.12
1                Pizza Place  0.12
2             Ice Cream Shop  0.06
3  Middle Eastern Restaurant  0.06
4         Chinese Restaurant  0.06


----Weston----
                    venue  freq
0           Train Station  0.13
1  Furniture / Home Store  0.07
2                   Diner  0.07
3                Pharmacy  0.07
4              Restaurant  0.07


----Wexford, Maryvale----
                       venue  freq
0  Middle Eastern Restaurant  0.09
1                Pizza Place  0.09
2               Burger Joint  0.06
3              Grocery Store  0.06
4               Intersection  0.06


----Willowdale, Newtonbrook----
                       venue  freq
0   

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhood_venues_sorted = pd.DataFrame(columns=columns)
neighbourhood_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhood_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Restaurant,Sandwich Place,Caribbean Restaurant
1,"Alderwood, Long Branch",Discount Store,Pizza Place,Pharmacy,Park,Intersection
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Pizza Place,Park,Coffee Shop,Mobile Phone Shop
3,Bayview Village,Bank,Gas Station,Grocery Store,Japanese Restaurant,Skating Rink
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Bank,Fast Food Restaurant
...,...,...,...,...,...,...
93,"Willowdale, Willowdale West",Pharmacy,Convenience Store,Grocery Store,Park,Butcher
94,Woburn,Coffee Shop,Park,Chinese Restaurant,Mobile Phone Shop,Indian Restaurant
95,Woodbine Heights,Coffee Shop,Park,Thai Restaurant,Pizza Place,Café
96,York Mills West,Restaurant,Park,Coffee Shop,Intersection,Chinese Restaurant


In [28]:
type(neighbourhood_venues_sorted)

pandas.core.frame.DataFrame

## 4. Cluster Neighborhoods

In [29]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 4, 4, 0, 0, 4, 0, 4, 0])

In [30]:
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
toronto_grouped_clustering

,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.024390,0.0,0.0,0.0,...,0.02439,0.0,0.0,0.0,0.0,0.0,0.02439,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
94,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
95,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0


In [31]:
# add clustering labels
neighbourhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [32]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
# toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merge = pd.merge(left=toronto_df, right=neighbourhood_venues_sorted, on='Neighbourhood', how='inner')
toronto_merge

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4,Park,Pharmacy,Bus Stop,Shopping Mall,ATM
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Men's Store,Lounge,Golf Course,Grocery Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Theater,Café,Park,Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Furniture / Home Store,Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Park,Hotel,Ramen Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
97,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,0,Coffee Shop,French Restaurant,Breakfast Spot,Italian Restaurant,Bank
98,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Park,Sushi Restaurant,Men's Store
99,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,4,Park,Coffee Shop,Pizza Place,Brewery,Fast Food Restaurant
100,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,4,Park,Eastern European Restaurant,Italian Restaurant,Shopping Mall,Ice Cream Shop


In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merge['Latitude'], toronto_merge['Longitude'], toronto_merge['Neighbourhood'], toronto_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [34]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 0, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,North York,0,Coffee Shop,Men's Store,Lounge,Golf Course,Grocery Store
2,Downtown Toronto,0,Coffee Shop,Theater,Café,Park,Restaurant
3,North York,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Furniture / Home Store,Restaurant
4,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Park,Hotel,Ramen Restaurant
6,Scarborough,0,Restaurant,Coffee Shop,Fast Food Restaurant,Trail,Bakery
7,North York,0,Restaurant,Japanese Restaurant,Coffee Shop,Pizza Place,Gym
8,North York,0,Restaurant,Japanese Restaurant,Coffee Shop,Pizza Place,Gym
10,Downtown Toronto,0,Coffee Shop,Gastropub,Café,Italian Restaurant,Japanese Restaurant
15,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Gastropub,Bakery
19,East Toronto,0,Pub,Coffee Shop,Pizza Place,Beach,Breakfast Spot


#### Cluster 2

In [35]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 1, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,Scarborough,1,Breakfast Spot,Playground,Burger Joint,Park,Italian Restaurant


#### Cluster 3

In [36]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 2, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
48,North York,2,Park,Pool,Zoo,Farm,Eastern European Restaurant


#### Cluster 4

In [37]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 3, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
38,Scarborough,3,Chinese Restaurant,Coffee Shop,Discount Store,Pizza Place,Fast Food Restaurant
51,North York,3,Coffee Shop,Convenience Store,Bakery,Chinese Restaurant,Mediterranean Restaurant
70,Etobicoke,3,Gas Station,Pizza Place,Intersection,Chinese Restaurant,Sandwich Place
77,Etobicoke,3,Pharmacy,Sandwich Place,Intersection,Bank,Mobile Phone Shop
78,Scarborough,3,Chinese Restaurant,Shopping Mall,Restaurant,Sandwich Place,Caribbean Restaurant
85,Scarborough,3,Chinese Restaurant,Pizza Place,Park,Hobby Shop,Gym
90,Scarborough,3,Chinese Restaurant,Pizza Place,Intersection,Bank,Bakery


#### Cluster 4

In [38]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 4, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,4,Park,Pharmacy,Bus Stop,Shopping Mall,ATM
5,Etobicoke,4,Pharmacy,Convenience Store,Bakery,Golf Course,Shopping Mall
9,East York,4,Brewery,Pizza Place,Gym / Fitness Center,Breakfast Spot,Fast Food Restaurant
11,North York,4,Grocery Store,Fast Food Restaurant,Italian Restaurant,Pizza Place,Park
12,Etobicoke,4,Pizza Place,Park,Hotel,Restaurant,Café
14,East York,4,Coffee Shop,Park,Thai Restaurant,Pizza Place,Café
16,York,4,Pizza Place,Park,Grocery Store,Coffee Shop,Convenience Store
17,Etobicoke,4,Coffee Shop,Fish & Chips Shop,College Rec Center,Liquor Store,Shopping Mall
18,Scarborough,4,Pizza Place,Restaurant,Bank,Fast Food Restaurant,Coffee Shop
21,York,4,Pharmacy,Pizza Place,Park,Mexican Restaurant,Grocery Store
